In [14]:
import requests
from faker import Faker
import random
import time
import pyodbc
from datetime import datetime

faker = Faker()

cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-B5O30HDH\SQLEXPRESS;'
                      'Database=bestbuyStagingAreadb;'
                      'Trusted_Connection=yes;')


def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y %I:%M', prop)

print(random_date("1/1/2023 1:30", "1/1/2024 4:50", random.random()))

06/14/2023 05:28


In [15]:
def getInsertData(pageSize, page):
    api_endpoint = 'https://api.bestbuy.com/v1/products'

    api_key = 'vGrtaHwUGfKD9wqg1pgF6fDm'

    params = {
        'apiKey': api_key,
        'format': 'json',
        'pageSize': pageSize,
        'page':page
    }
    
    try:
        response = requests.get(api_endpoint, params=params)
        if response.status_code == 200:
            data = response.json()
            cursor = cnxn.cursor()
            for p in data['products']:
                
                for i in range(1,random.randrange(60,512)):
                    #Customer info:
                    p['customerName'] = faker.name()
                    p['customerAge'] = random.randrange(18,65)
                    p['customerGender'] = random.choice(['Female', 'Male'])
                    p['customuerEmail'] = faker.email()
                    p['customerCountry'] = faker.country()

                    #Sale info :
                    p['saleDate'] = random_date("1/1/2023 1:30", "1/1/2024 4:50", random.random())
                    #p['saleTime'] = p['saleDate'].split(' ')[1]
                    p['quantitySold'] = random.randrange(1,16)
                    p['paymentMethod'] = random.choice(['Credit cards', 'Debit cards','Paypal','Apple Pay','Amazon Pay','Google Pay'])
                    p['totalPrice'] = p['quantitySold'] * p['salePrice']
                    
                    #Insertion data into database
                    query = '''INSERT INTO Sales(sku,name,type,startDate,new,active,activeUpdateDate,regularPrice,salePrice,
                            priceUpdateDate,digital,preowned,url,productTemplate,customerReviewCount,
                            customerReviewAverage,customerTopRated,format,freeShipping,shippingCost,
                            specialOrder,shortDescription,class,subclass,department,mpaaRating,image,
                            condition,genre,customerName,customerAge,customerGender,customuerEmail,
                            customerCountry,saleDate,quantitySold,paymentMethod,totalPrice) 
                            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
                    
                    cursor.execute(query,p['sku'],p['name'],p['type'],p['startDate'],p['new'],p['active'],
                                   p['activeUpdateDate'],p['regularPrice'],p['salePrice'],p['priceUpdateDate'],
                                   p['digital'],p['preowned'],p['url'],p['productTemplate'],p['customerReviewCount'],
                                   p['customerReviewAverage'],p['customerTopRated'],p['format'],p['freeShipping'],p['shippingCost'],
                                   p['specialOrder'],p['shortDescription'],p['class'],p['subclass'],p['department'],p['mpaaRating'],p['image'],
                                   p['condition'],p['genre'],p['customerName'],p['customerAge'],p['customerGender'],p['customuerEmail'],
                                   p['customerCountry'],datetime.strptime(p['saleDate'],'%m/%d/%Y %H:%M'),p['quantitySold'],p['paymentMethod'],p['totalPrice'])#,p['saleTime'] ,saleTime
                cursor.commit()
            cursor.close()
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return -1

    except requests.RequestException as e:
        print(f"Request failed: {e}")

page = 28

while(True):
    getInsertData(pageSize=100,page=page)
    print(f'Data of page {page} inserted!')
    if page < 30:
        page = page + 1
    else:
        break
    time.sleep(4)


In [5]:
api_key = 'vGrtaHwUGfKD9wqg1pgF6fDm'
sku = 1000006
url_details = f"https://api.bestbuy.com/v1/products/{sku}.json?apiKey={api_key}"
response_details = requests.get(url_details)

if response_details.status_code == 200:
    product_details = response_details.json()
    print(product_details)
else:
    print('Error fetching...')

{'sku': 1000006, 'score': None, 'productId': None, 'name': 'Spy Kids: All the Time in the World [Includes Digital Copy] [Blu-ray] [2011]', 'source': None, 'type': 'Movie', 'startDate': '2011-04-26', 'new': False, 'active': True, 'lowPriceGuarantee': True, 'activeUpdateDate': '2020-02-29T03:42:41', 'regularPrice': 14.99, 'salePrice': 14.99, 'clearance': False, 'onSale': False, 'planPrice': None, 'priceWithPlan': [], 'contracts': [], 'priceRestriction': None, 'priceUpdateDate': '2023-01-25T00:02:01', 'digital': False, 'preowned': False, 'carriers': [], 'planFeatures': [], 'devices': [], 'carrierPlans': [], 'technologyCode': None, 'carrierModelNumber': None, 'earlyTerminationFees': [], 'monthlyRecurringCharge': '', 'monthlyRecurringChargeGrandTotal': '', 'activationCharge': '', 'minutePrice': '', 'planCategory': None, 'planType': None, 'familyIndividualCode': None, 'validFrom': None, 'validUntil': None, 'carrierPlan': None, 'outletCenter': None, 'secondaryMarket': None, 'frequentlyPurchas